# Fetch model of a specified Run

In [23]:
from zenml.post_execution import get_pipelines

In [ ]:
pipelines = get_pipelines()

pipelines

In [25]:
from zenml.post_execution import get_pipeline

# This way you can get a pipeline by name
pipeline_x = get_pipeline(pipeline="pytorch_experiment_tracking_pipeline")

In [26]:
from zenml.post_execution import get_pipeline

pipeline = get_pipeline(pipeline="pytorch_experiment_tracking_pipeline")
last_run = pipeline.runs[0]
last_step = last_run.steps[-1]

In [27]:
model = last_step.outputs["trained_model"].read()
print(model)

LinearRegression(
  (linear): Linear(in_features=16, out_features=1, bias=True)
)


In [28]:
test_acc = last_step.outputs["test_acc"].read()
print(f'{test_acc:.10f}')

0.0000575309


# Test fetched model

In [29]:
import pandas as pd
import torch


data = pd.read_csv('./data/data.txt', header=None, delim_whitespace=True)
features = pd.read_csv('./data/Features.txt', header=None)
columns = features.values.squeeze()
dictionary_columns = {x.split(' - ')[0]: x for x in columns}

In [30]:
from torch.utils.data.dataset import Dataset

class CustomDatasetFromCSV(Dataset):
    def __init__(self, csv_path):
        self.data = pd.read_csv(csv_path, header=None, delim_whitespace=True)
        x = self.data.iloc[:,:-2].values
        y = self.data.iloc[:,-1].values

        x = torch.tensor(x, dtype =torch.float32)
        self.x_train = torch.nn.functional.normalize(x, p=1.0, dim=1, eps=1e-12, out=None)

        self.y_train = torch.tensor(y, dtype =torch.float32).unsqueeze(1)

    def __len__(self):
        return len(self.x_train)
    
    def __getitem__ (self,idx):
        return self.x_train[idx], self.y_train[idx]


In [31]:
aaa= CustomDatasetFromCSV('./data/data.txt')

In [32]:
from torch.utils.data import DataLoader

train_size = int(0.8 * len(aaa))
test_size = len(aaa) - train_size
train_dataset, test_dataset = torch.utils.data.random_split(aaa, [train_size, test_size])


train_dataloader = DataLoader(train_dataset, batch_size=8)
test_dataloader = DataLoader(test_dataset, batch_size=8)

In [ ]:
device = "cuda" if torch.cuda.is_available() else "cpu"


with torch.no_grad():
    for id in range(len(test_dataset)):
        X, y = test_dataset[id]
        X, y = X.to(device), y.to(device)
        pred = model(X)
        p = pred.item()
        gt = y.item()
        print(f'prediction: {p:.3f}, gt: {gt:.3f}, difference: {p - gt:.3f}')